# RVC 1, Ch2.2
https://petercorke.github.io/spatialmath-python/func_3d.html

In [ ]:
# Works best with jupyter-notebook
##TODO: update plotting methods everywhere. outdated.

In [ ]:
%matplotlib notebook 
#%matplotlib widget 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import numpy as np

from spatialmath import *
from spatialmath.base import * # many methods angle2R type, core math funcs, plotting, etc...

import matplotlib.pyplot as plt
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

## Lec 3.12 Quaternions
https://petercorke.github.io/spatialmath-python/func_quat.html

## Quaternions

A quaternion is often described as a type of complex number but it is more useful (and simpler) to think of it as an order pair comprising a scalar and a vector.  

Let's create a 90 degree rotation about the x-axis: pass a list as [s vx vy vz]

Quickly visualize them here https://quaternions.online/

In [ ]:
q1 = Quaternion([0.707,0.707,0,0])
print(f'The quaternion is: {q1}')
print(f'The scalar portion is {q1.A[0]}')
print(f'The vector portion is {q1.A[-3:]}')
type(q1)

Where: 
- the scalar part corresponds to the first item in the array
- the vector corresponds to the last 3 numbers 

**Quaternion Class**:

*Note, these are not yet unit quaternions*.

Properties:
- s   returns sclar portion
- v   returns vector portion
- vec returns a numpy array for the whole quat

Methods:
- conj returns the conjugate quaternion
- unit return a unit quaternion if it is not one.
- norm returns the norm: the magnitude equivalent to the 4-vector.

Properties:

In [ ]:
q1.s

In [ ]:
q1.v

In [ ]:
q1.vec

Methods:

In [ ]:
q1.conj()

In [ ]:
q1.unit()

In [ ]:
q1.norm()

Operations:
Let's examine the types of operations we can perform with quaternions. First create a second quaternion.

We can create a second rotation of 90 degrees about the x-axis

In [ ]:
q2 = Quaternion([0.707,0.707,0,0])
q2

Operations like additions and subtraction are also possible. But their physical meaning is more nuanced. For example: https://gamedev.stackexchange.com/questions/121021/is-adding-quaternions-a-useful-operation

In [ ]:
q1 + q2

In general, you will compose rotations by multiplying them via the * operator. The operation is the hamilton product: \
https://en.wikipedia.org/wiki/Quaternion#Hamilton_product

The product of two rotation quaternions will be equivalent to the rotation q2 followed by the rotation q1. 

In [ ]:
q1*q2

In the above example, we get a 180 degree rotation about the x-axis.... Can you test other rotations? Order matters.

Multiplication can also be performed as the linear algebraic product of one quaternion converted to a 4x4 matrix.

Matrix representations are not equivalent to the rotation matrices we saw before. Note:\
https://en.wikipedia.org/wiki/Quaternion#Matrix_representations

In [ ]:
q1 = Quaternion([0.707,0,0,0.707]) # 90 degree rotation by z.
q1.matrix

and the other as a 4-vector 

In [ ]:
q1.matrix @ q2.vec # 90 degree z-rotation, followed by a 90 degree x-rotation.

The product of a quaternion and its conjugate is a **scalar** equal to the square of its norm

In [ ]:
q1 * q1.conj()

Conversely, a quaternion with a zero scalar part is called a _pure quaternion_

In [ ]:
Quaternion.Pure([1, 2, 3])

### Unit quaternions

A quaternion with a unit norm is called a _unit quaternion_ .  

It is a group and its elements represent **rotation in 3D space**.  It is in all regards like an $\mbox{SO}(3)$ matrix except for a _double mapping_ -- a quaternion and its element-wise negation represent the same rotation.

In [ ]:
q1 = UnitQuaternion.Rx(60, 'deg')
print(q1)

print('\n')
print(f'The cosine of 30 degrees is: {np.cos( np.deg2rad(30) )}' )
print(f'The cosine of 30 degrees is: {np.sin( np.deg2rad(30) )}' )

Quaterions can be easily plot by calling the plot() method.

In [ ]:
help(q1.plot)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
q1.plot(block=False, dims=[-1,1],color='green', frame='q1', length=0.75, style='arrow', projection='ortho')

The convention is that unit quaternions are denoted using double angle brackets.  The norm, as advertised is indeed one

In [ ]:
q1.norm()

Let's test unit quaternion multiplication.

In [ ]:
q1 = UnitQuaternion.Rx(45, 'deg')
q2 = UnitQuaternion.Rz(90, 'deg')

Rotations can be composed by quaternion multiplication. 

In [ ]:
q3 = q1 * q2
q3

Which again is applied in reverse... First, the 90 degree rotation about z, followed by the 45 degree about the original x-axis. Visualize it.

In [ ]:
q3.plot(block=False, dims=[-1,1],color='blue', frame='q3', length=0.75, style='arrow', projection='ortho')

We can convert a Unit quaternion to a **rotation matrix** with the property R

In [ ]:
q3.R

which yields exactly the same answer as if we'd done it using SO(3) rotation matrices

In [ ]:
SO3.Rx(45, 'deg') * SO3.Rz(90, 'deg')

The advantages of unit quaternions are that

1. they are compact, just 4 numbers instead of 9
2. multiplication involves fewer operations and is therefore faster
3. numerical errors build up when we multiply rotation matrices together many times, and they lose the structure (the columns are no longer unit length or orthogonal).  Correcting this, the process of _normalization_ is expensive.  

For unit quaternions errors will also compound, but normalization is simply a matter of dividing through by the norm

In [ ]:
print(f'q3 is:\n{q3}\n')
print(f'It\'s norm is: {q3.norm()}\n')
print(f'The normalized quaternion is:\n{q3/q3.norm()}')

### Unit quaternions Inverses

In [ ]:
q1 = UnitQuaternion.Rx(60, 'deg')

In [ ]:
q1

We can compute the inverse in two ways:

    - use the inv() method
    - use the / operator.

In [ ]:
q1 * q1.inv()

or

In [ ]:
q1 / q1

### Converting Quaternions to S03 objects

We can convert any unit quaternion to an SO3 object if we wish

In [ ]:
q1.SO3()

and conversely, any `SO3` object to a unit quaternion

In [ ]:
UnitQuaternion( SO3.Rx(60, 'deg'))

A unit quaternion is not a minimal representation. 

Since we know the magnitude is 1, then with any 3 elements we can compute the fourth upto a sign ambiguity. 
Recall how to extract the vector comopnent of the quaternion:

In [ ]:
print(q1.vec3)
print(q1.v)

You can now take two quaternions and multiply their vector components

In [ ]:
a = UnitQuaternion.qvmul( q1.vec3, q2.vec3)
a

From which we can recreate the unit quaternion by computing unit quantities.

In [ ]:
UnitQuaternion.Vec3(a)